我觉得图的题目可以不严谨的分成两类。     
* 第一类是DAG。   
* 第二类不是DAG。    
DAG的意思是Directed Acyclic Graph, 即有向无环图。    
有向无环的好处是，我们访问过一个点之后，肯定不会再回来了，所以我们不需要用一个set来标记是否访问过某个点。    
这样就使得我们的代码特别好写。     

# 1. DAG

DAG 有很多种变形，比如：
* Tree/Binary Tree/Binary Search Tree
* M by N Matrix, only move to certain directions
* Array/List, move to the end
* and so on.

## 1.1 Tree/Binary Tree/Binary Search Tree
略, 参见树相关的笔记。

## 1.2 M by N Matrix, only move to certain directions

## 1.3 Array/List, move to the end

比如
permutations

- [055: Jump-Game](https://leetcode.com/problems/jump-game/)


图的算法应该有四大考点, 很难超出这个范围之外了。     
1. 简单BFS和DFS 
2. Topological Sort 
3. Union Find
4. Dijkstra    

基础操作，首先是如何建图。     
建图包括，有向图，无向图，加权图。当然UnionFind情况不太一样     
建完图包括

不错的简单BFS 和 DFS 的题       
* [417: Pacific-Atlantic-Water-Flow](https://leetcode.com/problems/pacific-atlantic-water-flow/)    
核心，从边缘的点开始搜索交叉的部分。      
* [310: Minimum-Height-Trees](https://leetcode.com/problems/minimum-height-trees/)         
    用 degrees 法，一层层找到并且删除 leaves
    
* [1306: Jump-Game-III](https://leetcode.com/problems/jump-game-iii/)     很简单的题

* [139: Word-Break](https://leetcode.com/problems/word-break/)
这个题是个 BFS 没想到吧

* [133: Clone-Graph](https://leetcode.com/problems/clone-graph/)   
由 133 想到的

###  BFS 和 DFS Iteration 的写法 
Iteration 我们是不好用 prev node 来追溯父节点的,        
所以如果我们需要进行一些父子节点的操作，      
我们必须在 把子节点加入队列之前就进行相应的操作。       
实在要追溯父节点的话，可以用一个 HashMap 追溯父节点。         

## 简单DFS和BFS

* [797: All-Paths-From-Source-to-Target](https://leetcode.com/problems/all-paths-from-source-to-target/)     
这个就是一个经典的DFS，这个题 BFS 不好做的。 DFS 是一个 backtracking, 事实上就是 postOrder.    
实际上也可以BFS。BFS的方法是我们直接丢一个 path 到 queue 中。 
        这个题注意由于我们是要 all possible path, 我们不能 mark vertex
        由于又是 acyclic graph 所以不用担心有环，因此不需要写 marked。
        如果是有环的情况，我们要 Mark edge 的    

* [785: Is-Graph-Bipartite?](https://leetcode.com/problems/is-graph-bipartite/)   
    判断图是否有环


因为一个图，不一定所有的都是联通的，那么我们需要一个     
for(int i = 0; i < N; i++) 来假设这个图每个点都不联通，来试试。    
如果已经访问过，那么直接跳过。在每个for循环里面，就是我们的一个简单的    
DFS，好比树的DFS一样。我们新建一个 Stack, 把它的儿子们/邻居 都加进去。   
BFS，把 stack 换成 queue。

In [8]:
public void DFS(int[][] graph) {
    int N = graph.length;
    boolean[] marked  = new boolean[N];

    for(int i = 0; i < N; i++ ) {
        if( marked[i] ) continue;

        Stack<Integer> stack = new Stack<>();
        stack.push(i);
        marked[i] = true;

        while(!stack.isEmpty()){
            Integer node = stack.pop();
            for(int neighbor : graph[node]) {
                if( ! marked[neighbor] ) {
                    stack.push(neighbor);
                    marked[neighbor] = true;
                }
            }
        }
    }
    return;
}




public void BFS(int[][] graph) {
    int N = graph.length;
    boolean[] marked  = new boolean[N];

    for(int i = 0; i < N; i++ ) {
        if( marked[i] ) continue;

        Queue<Integer> queue = new LinkedList<>();
        queue.offer(i);
        marked [i] = true;

        while(!queue.isEmpty()){
            int size = queue.size();
            while(size > 0) {
                size--;
                Integer node = queue.poll();   // DFS 是可以在这里mark的，然而BFS 是不能等到在这里才mark的
                for(int neighbor : graph[node]) {
                    if( !marked [neighbor]){
                        queue.offer(neighbor);
                        marked [neighbor] = true;  // BFS 只能在这里 mark 
                    }
                }
            }
        }
    }
    return;
}

上面的图是一个已经建立好的图，如果图没有建立好，那么我们需要首先建图。
无向图的建立和有向图的建立差不多。       
建图我们一般用 Map，某些情况可以用ArrayList<Integer>[] graph = new ArrayList[N];

下面是 200 题，我们用它来说明为什么 DFS/BFS 在入 stack/queue 的时候就 应该mark它         
如果是访问的时候才 mark 的话，我们会陷入循环中。         
两个 comment 掉的语句交换一下，这个解法就 TLE 了。       
因为 DFS/BFS的 stack/queue 可能很长，当我们还没有访问到 stack/queue 中一些储存的节点的时候，把它们又加入了一遍。
当然这种情况，说明我们的图不是 DAG. 对于 DAG，当然没有这些问题了。      


```java
class Solution {
    private int rows;
    private int cols;
    private char[][] grid;
    private boolean[][] marked;
    private final int[][] DS = new int[][]{{1,0},{0,1},{-1,0},{0,-1}};
    
    public int numIslands(char[][] grid) {
        this.rows = grid.length;
        if(rows == 0){
            return 0;
        }
        this.cols = grid[0].length;
        this.grid = grid;
        this.marked = new boolean[rows][cols];
        
        int count = 0;
        
        for(int i = 0; i < rows; i++ ){
            for(int j = 0; j < cols; j++){
                if( marked[i][j] || grid[i][j] == '0')  continue;
                count+=1;
                LinkedList<int[]> queue = new LinkedList<>();
                marked[i][j] = true;
                queue.offer(new int[]{i,j});
                while(!queue.isEmpty()){
                    int size = queue.size();
                    while(size > 0 ){
                        size -= 1;
                        int[] cur = queue.poll();
                        int row = cur[0];
                        int col = cur[1];
                        
//                         marked[row][col] = true; //这里mark 不好

                        for(int[] next : DS ){
                            int r = row + next[0];
                            int c = col + next[1];
                            if( r < 0 || c < 0 || r >= rows || c >= cols || grid[r][c] == '0'){
                                continue;
                            }
                            if( marked[r][c] ) {
                                continue;
                            }
                            marked[r][c] = true; // 在入queue前 就mark
                            queue.offer(new int[]{r,c});
                        }
                    }
                }
            }
        }
        return count;
    }
}
```

## DFS Backtracking
例题：[797: All-Paths-From-Source-to-Target](https://leetcode.com/problems/all-paths-from-source-to-target/)
### Recursive
```java
class Solution {
    private List<List<Integer>> ans = new ArrayList<>();
    private LinkedList<Integer> temp = new LinkedList<>();
    private int end;
    private int[][] graph;
    public List<List<Integer>> allPathsSourceTarget(int[][] graph) {
        this.end = graph.length - 1;
        this.graph = graph;
        temp.add(0);
        dfs(0);
        return ans;
    }
    public void dfs(int cur) {
        if( cur == end ) {
            ans.add(new ArrayList<>(temp));
            return;
        }
        for(int next : graph[cur]) {
            temp.addLast(next);
            dfs(next);
            temp.removeLast();
        }
    }
}
```


## 是否有环？

###  无向图 有环？
无向图判断是否有环。首先你要分析无向图是什么，无向图其实是相当于两个vertex互相指，有两个有向edge。    
那么我们找环的依据，就是如果我们在dfs中，当前的 vertex 的 下一个 vertex 不是我们的上一个 vertex 的话，  
说明我们就遇到环了。     
这里以   
[261: Graph-Valid-Tree](https://leetcode.com/problems/graph-valid-tree/)    
为例，说明了无向图如何建图，以及找环。

```java
class Solution {
    // Graph是Tree，首先要只有一个部分，其次不能有环。
    // count 数有几个部分
    // hasCycle 判断是否有环
    private int count;
    private boolean hasCycle;
    private boolean[] visited;
    
    public boolean validTree(int n, int[][] edges) {
        // Initialize Global Variables
        this.count = 0;
        this.visited = new boolean[n];
        this.hasCycle = false;
        // Initialize Graph
        List<Integer>[] adj = new ArrayList[n];
        for(int i = 0; i < n; i++) adj[i] = new ArrayList<>();
        for(int[] pairs : edges){
            adj[pairs[0]].add(pairs[1]);
            adj[pairs[1]].add(pairs[0]);
        }
        // DFS
        for(int i = 0; i < n; i++){
            if(!this.visited[i]){
                dfs(adj,i,i);
                count++;
            }
        }
        // Graph是Tree，首先要只有一个部分，其次不能有环。
        return (count < 2) && (!hasCycle) ;
    }
    
    private void dfs(List<Integer>[] adj, int cur, int last ){
        visited[cur] = true;
        for(int next : adj[cur]){
            if(!visited[next]){
                dfs(adj, next, cur);
                continue;
            }
            if(next != last){
                this.hasCycle = true;
            }
        }
    } 
}
```

### 有向图 有环？
判断有向图是否有环，那么就是判断一个图是否是DAG(有向无环图)。        

### 拓扑排序
例题有      
[207: Course-Schedule](https://leetcode.com/problems/course-schedule/)      
[210: Course-Schedule-II](https://leetcode.com/problems/course-schedule-ii/)   

拓扑排序的前提是：有向无环图。       
一般的解题方法有两种，DFS，和，BFS。BFS严格来说不是拓扑排序了。

#### DFS 拓扑排序    
DFS拓扑排序其实就是记录 reverse post order.    
Why it works: Each vertex v gets added to the end of the postorder list only after considering all descendants of v. Thus, when any v is added to the postorder list, all its descendants are already on the list. Thus reversing this list gives a topological ordering.                

严格的拓扑排序，首先要用DFS判断是否是无环图。这个只需要做一点点改动就可以，非常简单。     
其方法是，添加一个额外的用于判断是否是在环内的集合 onStack。该集合可以用Set, 也可以用 boolean[],   
做的是类似于 backtracking. 如果我们在一次DFS中就再次遇到了 onStack 中的元素，那么我们就可以断定  
有环！当然每次dfs完，记得从onStack 中把标记归到原位。如 onStack[vertex] = false; 这样的一个  
PostOrder操作就很好。


```java
private boolean[] marked;
private boolean[] onStack;
private boolean hasCycle;
private ArrayList<Integer>[] graph;

public boolean graphHasCycle(int numCourses, int[][] prerequisites) {
    this.marked = new boolean[numCourses];
    this.onStack = new boolean[numCourses];
    this.hasCycle = false;
    this.graph = new ArrayList[numCourses];
    
    // 建图
    for(int i = 0; i < numCourses; i++ ) {
        graph[i] = new ArrayList<Integer>();
    }

    for(int[] edge : prerequisites) {
        graph[edge[1]].add(edge[0]);
    }
    // DFS
    for(int i = 0; i < numCourses; i++ ) {
        if(!marked[i]) {
            dfs(i);
        }
    }
    return hasCycle;
}

public void dfs(int vertex) {
    marked[vertex] = true;
    onStack[vertex] = true;

    for(int neighbor : graph[vertex] ) {
        if( hasCycle ) {
            return;
        }
        if(!marked[neighbor]) {
            dfs(neighbor);
        }else if( onStack[neighbor] ) {
            hasCycle = true;
            return;
        }
    }
    onStack[vertex] = false;
}
```


如果确定是有向无环图，那么**拓扑排序的顺序就是一个简单的DFS的postOrder的逆序输出**。   
以这个 course schedule ii find order 为例，我们就是多加了一个dfs的  
post order 的记录。我们用了三种不一样的方式来记录，都在leetcode 提交过了，是等价的。    
代码如下  

```java
class Solution {
    private boolean[] marked;
    private boolean[] onStack;
    private boolean hasCycle;
    private ArrayList<Integer>[] graph;
    
    private Stack<Integer> reversePost;
    private ArrayList<Integer> postList;
    private LinkedList<Integer> reversePostList;
    
    public int[] findOrder(int numCourses, int[][] prerequisites) {
        
        this.marked = new boolean[numCourses];
        this.onStack = new boolean[numCourses];
        this.hasCycle = false;
        this.graph = new ArrayList[numCourses];
        
        this.reversePost = new Stack<>();
        this.postList = new ArrayList<>();
        this.reversePostList = new LinkedList<>();
        
        for(int i = 0; i < numCourses; i++ ) {
            graph[i] = new ArrayList<Integer>();
        }
        
        for(int[] edge : prerequisites) {
            graph[edge[1]].add(edge[0]);
        }
        
        for(int i = 0; i < numCourses; i++ ) {
            if(!marked[i]) {
                dfs(i);
            }
        }
        if(hasCycle) return new int[0];
        
        int[] ansRP = new int[numCourses];
        int[] ansPList = new int[numCourses];
        int[] ansRPList = new int[numCourses];
        
        for(int i = 0; i < numCourses; i++) {
            ansRP[i] = reversePost.pop();
            ansPList[i] = postList.get(numCourses - i - 1);
            ansRPList[i] = reversePostList.get(i);
        }
        
        return ansRPList;
    }
    
    public void dfs(int vertex) {
        marked[vertex] = true;
        onStack[vertex] = true;
        for(int neighbor : graph[vertex] ) {
            if( hasCycle ) {
                return;
            }
            
            if(!marked[neighbor]) {
                dfs(neighbor);
            }else if( onStack[neighbor] ) {
                hasCycle = true;
                return;
            }
        }
        onStack[vertex] = false;
        
        
        reversePost.push(vertex);
        postList.add(vertex);
        reversePostList.addFirst(vertex);
    }
}
```

#### BFS 拓扑排序   
BFS 的关键是 入度表，入度即一个 vertex 有多少个爹。     
其逻辑是，我们先找到所有的没有爹的vertex，以这些作为起点进行BFS。确实也很好理解，没有爹的vertex确实是拓扑排序最早入列的。        
对于每个没有爹的点，我们BFS他们的子女的时候，将他们的子女的爹数减1，一旦因此出现了无父无母的子女，我们就把他们加入到BFS的队列之中。    
这样的顺序就是拓扑排序了。如果BFS能够遍历所有的顶点，说明无环。    
补充一下，如果有环，那么环里面的元素就是互为爹妈。那么永远都加不到我们的queue当中去。         

In [7]:
// 以 207, 210 为例， 我们可以使用 queue bfs， 但是这样我们就丢掉了顺序的信息。
// 所以为了记录顺序，我们必须另外记录一个顺序。如果我们就用 ArrayList BFS，那么BFS的顺序也被我们记录了。
public int[] findOrder(int numCourses, int[][] prerequisites) {
    if(numCourses == 0) return new int[0];
    int[] inDegrees = new int[numCourses];

    // 建图
    ArrayList<Integer>[] graph = new ArrayList[numCourses];
    for(int i = 0 ; i < numCourses; i++) {
        graph[i] = new ArrayList<Integer>();
    }

    for(int[] pair : prerequisites) {
        graph[pair[1]].add(pair[0]);
        inDegrees[pair[0]] += 1;
    }

    // BFS
    ArrayList<Integer> bfs = new ArrayList<>();

    for(int i = 0 ; i < numCourses; i++ )  {
        if( inDegrees[i] == 0) {
            bfs.add(i);
        }
    }


    for(int i = 0; i < bfs.size(); i++) {
        for(int childVertex :  graph[bfs.get(i)]) {
            inDegrees[childVertex] -= 1;
            if(inDegrees[childVertex] == 0) {
                bfs.add(childVertex);
            }
        }
    }

    // 产生答案
    if(bfs.size() == numCourses) {
        int[] ans = new int[numCourses];
        for (int i = 0; i < numCourses; i++) {
            ans[i] = bfs.get(i);
        }
        return ans;
    }
    return new int[0];
}

## Dijkstra   

* [743: Network-Delay-Time](https://leetcode.com/problems/network-delay-time/)     
* [787: Cheapest-Flights-Within-K-Stops](https://leetcode.com/problems/cheapest-flights-within-k-stops/)

## Eulerian Path 
找最小欧拉路径，欧拉路径是遍历所有的边。
* [332: Reconstruct-Itinerary](https://leetcode.com/problems/reconstruct-itinerary/)